In [2]:
import torch
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import time
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from typing import Iterable
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from joblib import dump, load

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch import Tensor
import sys

In [3]:
site = 'B'

In [4]:
# load the data
data_finetune = pd.read_csv("data_finetune.csv", index_col = 0)

In [5]:
data_finetune

,ts,Month,Day,Hour,instanceID,Wind_speed,TI,Temperature,Power
450954,2020-12-13 23:00:00,12,13,23,B_WTG01,6.219789,13.680036,9.128333,542.963406
250695,2020-07-12 10:30:00,7,12,10,B_WTG01,5.502723,19.909536,18.020833,361.234628
471681,2020-12-29 22:50:00,12,29,22,B_WTG01,6.885329,12.528561,4.351667,762.492368
408888,2020-11-11 12:00:00,11,11,12,B_WTG01,9.122236,10.780250,10.756667,1569.216417
14913,2020-01-12 12:10:00,1,12,12,B_WTG01,5.227403,18.879639,8.642500,280.115814
...,...,...,...,...,...,...,...,...,...
425465,2020-11-24 06:50:00,11,24,6,B_WTG09,7.298661,15.227533,12.595834,781.406852
101672,2020-03-19 10:40:00,3,19,10,B_WTG09,3.191359,17.653836,9.745000,40.664581
42776,2020-02-03 00:00:00,2,3,0,B_WTG09,6.644093,14.493794,9.076667,612.544883
9908,2020-01-08 15:20:00,1,8,15,B_WTG09,5.745972,13.970354,7.398333,342.725793


In [6]:
# load normalization function 
scaler1 = load('scaler1.bin')
scaler2 = load('scaler2.bin')

In [7]:
turbine_count = data_finetune['instanceID'].nunique()

### Define network

In [8]:
class Net(nn.Module):
    def __init__(self, dims: Iterable[int], output_activation: nn.Module = None):
        """Creates a network using ReLUs between layers and no activation at the end

        :param dims (Iterable[int]): tuple in the form of (IN_SIZE, HIDDEN_SIZE, HIDDEN_SIZE2,
            ..., OUT_SIZE) for dimensionalities of layers
        :param output_activation (nn.Module): PyTorch activation function to use after last layer
        """
        super().__init__()
        self.input_size = dims[0]
        self.out_size = dims[-1]
        self.layers = self.make_seq(dims, output_activation)

    @staticmethod
    def make_seq(dims: Iterable[int], output_activation: nn.Module) -> nn.Module:
        """Creates a sequential network using ReLUs between layers and no activation at the end

        :param dims (Iterable[int]): tuple in the form of (IN_SIZE, HIDDEN_SIZE, HIDDEN_SIZE2,
            ..., OUT_SIZE) for dimensionalities of layers
        :param output_activation (nn.Module): PyTorch activation function to use after last layer
        :return (nn.Module): return created sequential layers
        """
        mods = []

        for i in range(len(dims) - 2):
            mods.append(nn.Linear(dims[i], dims[i + 1]))
            mods.append(nn.ReLU())

        mods.append(nn.Linear(dims[-2], dims[-1]))
        if output_activation:
            mods.append(output_activation())
        return nn.Sequential(*mods)
    
    def forward(self, x: Tensor) -> Tensor:
        """Computes a forward pass through the network

        :param x (torch.Tensor): input tensor to feed into the network
        :return (torch.Tensor): output computed by the network
        """
        # Feedforward
        return self.layers(x)


In [9]:
def train(X, y, quantile, net, lr, batch_size, epoch):    
    
    # create tensor dataset
    train = TensorDataset(Tensor(X), Tensor(y))

    # create data loader from dataset
    trainset = DataLoader(train, batch_size = batch_size, shuffle = True)

    # define optimizer
    optimizer = optim.Adam(net.parameters(), lr = lr)
        
    mse_loss = nn.MSELoss()

    for ep in range(epoch):

        for t in trainset:
            X_temp, y_temp = t
            output = net(X_temp)
            residual = y_temp - output
            loss = Tensor.max(quantile*residual, (quantile-1)*residual).mean()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
    return net, net.state_dict()

### Fine tuning

use a very low learning rate at this stage, because we are training on a dataset that is very small. This is to prevent the risk of overfitting very quickly if we apply large weight updates. 

In [10]:
# load hyperparameters
model = torch.load(sys.path[0] + '/hparams_finetune.pth')

# load the pretrained weights
pretrain = torch.load(sys.path[0] + '/pretrain.pth')

In [11]:
pretrain

{'median': OrderedDict([('layers.0.weight',
               tensor([[-0.4231,  0.2336, -0.0885],
                       [-0.6750, -0.1923,  0.1808],
                       [-0.0807, -0.1944,  0.1123],
                       [ 0.0126, -0.2398, -0.1289],
                       [ 0.4486, -0.2861,  0.3437],
                       [-0.6428,  0.3671, -0.4600],
                       [ 0.5408, -0.2385, -0.1524],
                       [ 0.3557, -0.1515,  0.2148],
                       [-0.6883, -0.1069,  0.0626],
                       [-0.4806,  0.2262,  0.1405],
                       [ 0.0316, -0.0122,  0.1909],
                       [ 0.0300,  0.3099,  0.2421],
                       [-0.3464, -0.2597, -0.2908],
                       [ 0.8016,  0.2263, -0.2149],
                       [ 0.3245,  0.2140,  0.3012],
                       [ 0.6756,  0.0734, -0.0814]])),
              ('layers.0.bias',
               tensor([-0.1946, -0.3724,  0.6272,  0.1927, -0.2128,  0.2722,  0.6633, -0.

In [12]:
# # define quantiles and hyperparameters
q_median = 0.5
q_upper = 0.975
q_lower = 0.025
dims = model['dims']
lr = model['lr']
batch_size = int(model['batch_size'])
epoch = model['epoch']

In [13]:
print("lr:", lr)
print("batch size:", batch_size)
print("epoch:", epoch)

lr: 0.001
batch size: 500
epoch: 20


In [17]:
%%time
#################################################### training ######################################################### 

turbines = data_finetune.instanceID.unique()
median_state_dict_all = []
UQ_state_dict_all = []
LQ_state_dict_all = []


for ID in turbines:
    
    # select data based on turbine ID
    data_temp = data_finetune[data_finetune['instanceID'] == ID]

    # normalize data
    X = scaler1.transform(data_temp.iloc[:, 5:-1])
    y = scaler2.transform(data_temp.iloc[:, -1:])
    
    # create network and load pretrain weights
    net_median_temp = Net(dims = dims)
    net_median_temp.load_state_dict(pretrain['median'])
    
    net_upper_temp = Net(dims = dims)
    net_upper_temp.load_state_dict(pretrain['UQ'])
    
    net_lower_temp = Net(dims = dims)
    net_lower_temp.load_state_dict(pretrain['LQ'])
    
    # train
    net_median_temp, median_state_dict = train(X=X, y=y, quantile=q_median, net=net_median_temp, 
                                          lr=lr, batch_size=batch_size, epoch=epoch)
    net_upper_temp, UQ_state_dict = train(X=X, y=y, quantile=q_upper, net=net_upper_temp, 
                                          lr=lr, batch_size=batch_size, epoch=epoch)
    net_lower_temp, LQ_state_dict = train(X=X, y=y, quantile=q_lower, net=net_lower_temp, 
                                          lr=lr, batch_size=batch_size, epoch=epoch)
    
    median_state_dict_all.append(median_state_dict)
    UQ_state_dict_all.append(UQ_state_dict)
    LQ_state_dict_all.append(LQ_state_dict)
    
    print('Done', ID)
    

Done B_WTG01
Done B_WTG02
Done B_WTG03
Done B_WTG04
Done B_WTG05
Done B_WTG06
Done B_WTG07
Done B_WTG08
Done B_WTG09
Wall time: 1min 50s


### Save trained network

In [18]:
median_state_dict_all_zip = dict(zip(turbines, median_state_dict_all))
UQ_state_dict_all_zip = dict(zip(turbines, UQ_state_dict_all))
LQ_state_dict_all_zip = dict(zip(turbines, LQ_state_dict_all))

In [19]:
# # save trained network

torch.save(median_state_dict_all_zip, sys.path[0] + '/median.pth')
torch.save(UQ_state_dict_all_zip, sys.path[0] + '/UQ.pth')
torch.save(LQ_state_dict_all_zip, sys.path[0] + '/LQ.pth')